In [1]:
import cv2
import mediapipe as mp

In [2]:
## initialize pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [3]:
import struct
def return_bytes(pose_results):
    lst = []
    for i in range(33):
        pt = pose_results.pose_landmarks.landmark[i]
        lst.append(pt.x)
        lst.append(pt.y)
        lst.append(pt.z)
        lst.append(pt.visibility)
    return struct.pack('%sf' % len(lst), *lst)

In [ ]:
import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 10000        # Port to listen on (non-privileged ports are > 1023)

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen()

conn, addr = s.accept()
print('Connected by', addr)

In [4]:
import socket

# Define server address and port
HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 11000        # The port used by the server

s =  socket.socket(socket.AF_INET, socket.SOCK_DGRAM)


In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    # read frame
    _, frame = cap.read()
    try:
        # resize the frame for portrait video
        frame = cv2.resize(frame, (600,480))
        # convert to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # process the frame for pose detection
        pose_results = pose.process(frame_rgb)
        # print(pose_results.pose_landmarks)

        #print(pose_results.pose_landmarks.landmark[0].x)
        #conn.send(return_bytes(pose_results))
        s.sendto(return_bytes(pose_results), (HOST, PORT))

        # draw skeleton on the frame
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        # display the frame
        cv2.imshow('Output', frame)
    except:
        break

    if cv2.waitKey(100) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()